In [ ]:
#calculate coherence
#not used in the final report

In [ ]:
import database.neuralynx_extract.combinato_tools as ct
import database.neuralynx_extract.nlxio as nlxio

import preprocessing.data_preprocessing.refractor_utils as ru
import sys

import matplotlib.pyplot as plt
import datajoint as dj
import numpy as np
import neurodsp.filt
import scipy
from pathlib import Path

# Local application imports 
from database.db_setup import *
import preprocessing.data_preprocessing.binning as binning
import preprocessing.data_preprocessing.create_vectors_from_time_points as create

import helper_func.helper_func as hf

In [4]:
patient_id = 46
session_nr = 1

#top_dir = '/media/alado/TOSHIBA_EXT/thesis/output'
top_dir = '/home/anastasia/epiphyte/anastasia/output'
folder = f'05-Hilbert_transform_{patient_id}'
filtering_type = ['theta', 'slow_gamma', 'fast_gamma']

fs = 32768
#The resulting sample rate is up / down times the original sample rate.
up = 1
down=32
fs_downs = (up/down)*fs
dt = 1/fs_downs      # sampling period/time/interval or time resolution, often denoted as T

times = np.linspace(-500, 1000, num = 1501)
time_zero_idx = np.where(times == hf.find_nearest(times, 0))[0][0]

df_patient_info = pd.read_csv(f'{top_dir}/{patient_id}_channel_info.csv')
dict_replace = {'LA1':'LAA1', 'LA2':'LAA2', 'LA3':'LAA3', 'LA4':'LAA4', 'LA5':'LAA5', 'LA6':'LAA6', 'LA7':'LAA7', 'LA8':'LAA8',
               'RA1':'RAA1', 'RA2':'RAA2', 'RA3':'RAA3', 'RA4':'RAA4', 'RA5':'RAA5', 'RA6':'RAA6', 'RA7':'RAA7', 'RA8':'RAA8',}
df_patient_info_new = df_patient_info.replace(dict_replace)
df_stim_info = pd.read_csv(f'{top_dir}/{patient_id}_df_stim_info.csv')
all_stim = np.unique(df_stim_info['stim_id'])
all_stim_name = np.unique(df_stim_info['stim_name'])
all_brain_areas = np.unique(df_patient_info['recording_site'])
#brain_areas = ['LAA', 'LAH', 'LEC','LMH', 'LPHC', 'RAA', 'RAH', 'REC', 'RMH', 'RPHC']
brain_areas = np.unique(df_patient_info['brain_area'])

n_channels = 80
n_stim_present = 10
n_stim = 42

     index  patient_id  session_nr  stim_index position  stim_id  \
0     1344          46           1           0      pre       19   
1     1345          46           1           1      pre        0   
2     1346          46           1           2      pre       13   
3     1347          46           1           3      pre       17   
4     1348          46           1           4      pre        7   
..     ...         ...         ...         ...      ...      ...   
835   2179          46           1         835     post       19   
836   2180          46           1         836     post       22   
837   2181          46           1         837     post       11   
838   2182          46           1         838     post        2   
839   2183          46           1         839     post       30   

            filename        stim_name  is_500_days paradigm          time  
0          paul2.jpg             Paul            1      scr  2.186932e+07  
1        alison1.jpg           

In [5]:
my_file = Path(f'{top_dir}/anastasia/output/01-filtered_and_downsampled/{patient_id}_downsampled_data_gamma.npy')
if my_file.is_file():
    print('Load downsampled gamma-filtered data file')
    downsampled_data_gamma_patient = np.load(my_file)
    timestamps_downs = np.load(f'{top_dir}/anastasia/output/01-filtered_and_downsampled/downsampled/{patient_id}_timestamps_downs.npy')
    
else:
    print('You need to create downsampled gamma-filtered data file')
    
my_file = Path(f'{top_dir}/anastasia/output/01-filtered_and_downsampled/downsampled/{patient_id}_downsampled_data.npy')
if my_file.is_file():
    print('Load downsampled gamma-filtered data file')
    downsampled_data_broadband_patient = np.load(my_file)
    timestamps_downs_broadband = np.load(f'{top_dir}/anastasia/output/01-filtered_and_downsampled/downsampled/{patient_id}_timestamps_downs.npy')
    
else:
    print('You need to create downsampled broadband signal data file')

Load downsampled gamma-filtered data file
Load downsampled gamma-filtered data file


In [7]:
times = np.linspace(-500, 1000, num = 1501)
dt = times[1] - times[0]                     #sampling interval
T = times[-1]                                #total time of the recording.
N = 1501                                     #number of sample points per trial
scale = 2 * dt**2 / T                        #scaling constant

#for i in len(df_patient_info['channel_name']):
i=0
ch = df_patient_info.loc[i,'channel_name']
ch_site = df_patient_info.loc[i,'recording_site']
epochs = np.load(f'{top_dir}/anastasia/output/02-epochs/broadband_2_sec_mean_filter/{patient_id}_epochs_channel-{ch}_broadband.npy')

#numbers here come from computing the spectrogram
ps_pre = np.zeros((len(all_stim),65,22))
ps_post = np.zeros((len(all_stim),65,22))

#for st in all_stim:
st=0
pre = df_stim_info.loc[(df_stim_info['position']=='pre') & (df_stim_info['stim_id']==st)]
pre = pre.reset_index(drop=True)
post = df_stim_info.loc[(df_stim_info['position']=='post') & (df_stim_info['stim_id']==st)]
pre_index = np.array(pre['stim_index'])
post_index = np.array(post['stim_index'])

pre_sp_array = np.zeros((len(pre_index),65,49))
post_sp_array = np.zeros((len(post_index),65,49))
for i in range(len(pre_index)):
    pre_epoch = epochs[pre_index[i], 500:]
    post_epoch = epochs[post_index[i], 500:]
    
    #Spectrogram
    #freq_pre,time_pre,pwr_pre = scipy.signal.spectrogram(pre_epoch,fs_downs, window='hanning', nperseg=50, nfft=128, noverlap=20, detrend=False, scaling='density', mode='psd')
    #freq_post,time_post,pwr_post = scipy.signal.spectrogram(post_epoch,fs_downs, window='hanning', nperseg=50, nfft=128, noverlap=20, detrend=False, scaling='density', mode='psd')
    
    # Compute the Fourier transforms
    xf = np.array([scipy.fft.rfft(x) for x in pre_epoch]) # ... for each trial in E1
    yf = np.array([scipy.fft.rfft(y) for y in post_epoch]) # ... and each trial in E2
    
    # Compute the spectra
    Sxx = scale * (xf * xf.conj()).mean(0)       # Spectrum of E1 trials
    Syy = scale * (yf * yf.conj()).mean(0)       # ... and E2 trials
    Sxy = scale * (xf * yf.conj()).mean(0)       # ... and the cross spectrum
    
    #pre_sp_array[i,:,:] = pwr_pre
    #post_sp_array[i,:,:] = pwr_post
    
    # Compute the coherence.
    cohr = abs(Sxy) / (sqrt(Sxx) * sqrt(Syy))

f = scipy.fft.rfftfreq(N, dt)                # Define a frequency axis.
plot(f, cohr.real)                           # Plot coherence vs frequency,
xlim([0, 50])                                # ... in a chosen frequency range,
ylim([0, 1])                                 # ... with y-axis scaled,
xlabel('Frequency [Hz]')                     # ... and with axes labeled.
ylabel('Coherence')
title('Coherence between two electrodes')
show()

#NOTE that freq_pre and freq_post arrays are identical

t = np.linspace(-500, 1000, time_pre.shape[0])

current_stim_index = pre.loc[0,'stim_index']
current_stim_name = pre.loc[0,'stim_name']
current_stim_paradigm = pre.loc[0,'paradigm']

IndexError: tuple index out of range